<a href="https://colab.research.google.com/github/neuralguy18/demo/blob/main/workingwithtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00


In [2]:
import torch
import tiktoken
from importlib.metadata import version
print("torch version:"+ version("torch"))
print("tiktoken version:"+ version("tiktoken"))

torch version:2.5.1+cu121
tiktoken version:0.8.0


In [3]:

import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7e375018acb0>)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
import re
text = "Hello, is this -- a test ?"
result = re.split(r'([.,:;?_!"()\']|--|\s)',text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'is', 'this', '--', 'a', 'test', '?']


In [6]:
import re
preprocessed = re.split(r'([.,:;?_!"()\']|--|\s)',raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [7]:
all_words = sorted(list((set(preprocessed))))
vocab_size = len(all_words)
print(vocab_size)

1130


In [8]:
vocab = {word: index for index, word in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
  print(item)
  if i == 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [9]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [10]:
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [11]:
the_text = tokenizer.decode(ids)
print(the_text)

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [12]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {word: index for index, word in enumerate(all_tokens)}
print(vocab)


{'!': 0, '"': 1, "'": 2, '(': 3, ')': 4, ',': 5, '--': 6, '.': 7, ':': 8, ';': 9, '?': 10, 'A': 11, 'Ah': 12, 'Among': 13, 'And': 14, 'Are': 15, 'Arrt': 16, 'As': 17, 'At': 18, 'Be': 19, 'Begin': 20, 'Burlington': 21, 'But': 22, 'By': 23, 'Carlo': 24, 'Chicago': 25, 'Claude': 26, 'Come': 27, 'Croft': 28, 'Destroyed': 29, 'Devonshire': 30, 'Don': 31, 'Dubarry': 32, 'Emperors': 33, 'Florence': 34, 'For': 35, 'Gallery': 36, 'Gideon': 37, 'Gisburn': 38, 'Gisburns': 39, 'Grafton': 40, 'Greek': 41, 'Grindle': 42, 'Grindles': 43, 'HAD': 44, 'Had': 45, 'Hang': 46, 'Has': 47, 'He': 48, 'Her': 49, 'Hermia': 50, 'His': 51, 'How': 52, 'I': 53, 'If': 54, 'In': 55, 'It': 56, 'Jack': 57, 'Jove': 58, 'Just': 59, 'Lord': 60, 'Made': 61, 'Miss': 62, 'Money': 63, 'Monte': 64, 'Moon-dancers': 65, 'Mr': 66, 'Mrs': 67, 'My': 68, 'Never': 69, 'No': 70, 'Now': 71, 'Nutley': 72, 'Of': 73, 'Oh': 74, 'On': 75, 'Once': 76, 'Only': 77, 'Or': 78, 'Perhaps': 79, 'Poor': 80, 'Professional': 81, 'Renaissance': 82, 'Ri

In [13]:
class SimpleTokenizerV2:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [item if item in self.str_to_int else "<|unk|>"
                    for item in preprocessed]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
    return text


In [14]:
text1 = "Hello, what do you like to have?"
text2 = "Nothing,thanks for asking"
text = "<|endoftext|> ".join((text1,text2))
print(text)

Hello, what do you like to have?<|endoftext|> Nothing,thanks for asking


In [15]:
tokenizer = SimpleTokenizerV2(vocab)
ids = tokenizer.encode(text)
ids

[1131, 5, 1089, 355, 1126, 628, 1016, 530, 10, 1130, 1131, 5, 1131, 456, 1131]

In [16]:
print(tokenizer.decode(ids))

<|unk|>, what do you like to have? <|endoftext|> <|unk|>, <|unk|> for <|unk|>


In [17]:
pip install tiktoken

In [19]:
from importlib.metadata import version
import tiktoken
print("torch version:"+ version("torch"))
print("tiktoken version:"+ version("tiktoken"))

torch version:2.5.1+cu121
tiktoken version:0.8.0


In [20]:
tokenizer = tiktoken.get_encoding("gpt2")


In [22]:
integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 644, 466, 345, 588, 284, 423, 30, 50256, 10528, 11, 27547, 329, 4737]


In [24]:
print(tokenizer.decode(integers))

Hello, what do you like to have?<|endoftext|> Nothing,thanks for asking


In [25]:
int = tokenizer.encode("Akwirw ier")
print(int)

[33901, 86, 343, 86, 220, 959]


In [29]:
for i in int:
  print(f"{i}: {tokenizer.decode([i])}")

33901: Ak
86: w
343: ir
86: w
220:  
959: ier


In [26]:
print(tokenizer.decode(int))

Akwirw ier


In [31]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
len(enc_text)

5145

In [32]:
enc_sample = enc_text[50:]

In [38]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(x,y)


[290, 4920, 2241, 287] [4920, 2241, 287, 257]


In [41]:
inputs = []
j = 0
for i in x:
  inputs.append(i)
  print(f"{inputs} -> {y[j]}")
  j += 1


[290] -> 4920
[290, 4920] -> 2241
[290, 4920, 2241] -> 287
[290, 4920, 2241, 287] -> 257


In [45]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(context, "-------->",desired)

[290] --------> 4920
[290, 4920] --------> 2241
[290, 4920, 2241] --------> 287
[290, 4920, 2241, 287] --------> 257


In [54]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]
  print(tokenizer.decode(context), "-------->",tokenizer.decode([desired]))


 and -------->  established
 and established -------->  himself
 and established himself -------->  in
 and established himself in -------->  a


In [60]:
import torch
from torch.utils.data import Dataset,DataLoader
class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids = []
    self.target_ids =[]
    token_ids = tokenizer.encode(txt)
    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]


In [61]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
  tokenizer = tiktoken.get_encoding("gpt2")
  dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
  return dataloader

In [84]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

dataloader = create_dataloader_v1(raw_text,batch_size=1,
                                    max_length=6,stride=1,shuffle=False)
data_iter= iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464, 1807, 3619]]), tensor([[ 367, 2885, 1464, 1807, 3619,  402]])]


In [85]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807, 3619,  402]]), tensor([[2885, 1464, 1807, 3619,  402,  271]])]


In [71]:
dataloader = create_dataloader_v1(raw_text,batch_size=8,
                                    max_length=4,stride=4,shuffle=False)
data_iter= iter(dataloader)
inputs,targets = next(data_iter)
print("Inputs:\n", inputs)
print("Targets:\n",targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [76]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [78]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))


tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [79]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(token_embedding_layer.weight.shape)

torch.Size([50257, 256])


In [80]:
max_length = 4
dataloader = create_dataloader_v1(raw_text,batch_size=8,
                                    max_length=max_length,stride=max_length,shuffle=False)
data_iter= iter(dataloader)
inputs,targets = next(data_iter)
print("Token IDs:\n", inputs)
print("Inputs shape:\n",inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape:
 torch.Size([8, 4])


In [81]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [82]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_ids = torch.arange(0, max_length)
pos_embeddings = pos_embedding_layer(pos_ids)
print(pos_embeddings.shape)

torch.Size([4, 256])


In [83]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
